In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk.stem.porter as porter
from def_overlap_funcs import filter_words, similarity_score

## Similarità definizioni
Si sono create, per ogni studente, quattro definizioni. Il task successivo richiedeva di valutare la similarità tra le definizioni, per ognuno dei concetti scelti.
Il primo output visibile nel notebook rappresenta la tabella con un concetto per colonna e nelle righe le definizioni pulite da stop words e punteggiatura e stemmatizzate (ovvero ridotte alla loro *radice*).

In [2]:
stemmer = porter.PorterStemmer()

In [3]:
data = pd.read_csv('TLN-definitions-23.csv', sep=';')
data = data.applymap(lambda s: filter_words(set(word_tokenize(s))))
data = data.applymap(lambda tokens: {stemmer.stem(t) for t in tokens})

data

,door,ladybug,pain,blurriness
0,"{close, divid, construct, two, room, passag, u...","{small, spot, six, red, fli, typic, black, ins...","{physic, feel, distress, mental}","{sight, focu}"
1,"{close, open}","{symbol, luck, cultur, good, red, black, insec...","{emot, physic, bad, hurt, feel}","{shapeless, definit, absenc, border}"
2,"{wall, close, hole, peopl, let, divid, door, r...","{color, fli, orang, red, black, insect, dot}","{consciou, experi, be, suffer, fell}","{someth, border, clearli, felt, ca, sensat, see}"
3,"{access, anoth, area, one, usabl}","{red, small, insect, back}","{concept, suffer, describ, live}","{lack, sharp}"
4,"{access, delimit, allow, structur, area}","{fli, small, insect, round}","{physic, discomfort, feel}","{characterist, lack, clariti, precis}"
5,"{move, passag, use, object, pass, block}","{small, color, beatl, fli, famili, tendenti, r...","{burn, unpleas, sensat}","{blur, everyth, eye, sensat, see}"
6,"{push, hing, mean, histor, iron, rotat, use, o...","{small, back, luck, consid, insectivor, natur,...","{psycholog, physic, natur, creatur, felt, dama...","{perceiv, physic, whether, certain, inabl, goa..."
7,"{wall, close, room, use, open, object, separ, go}","{small, luck, cultur, red, black, insect, dot,...","{emot, injuri, caus, physic, sad, pain, feel}","{unfocus, memori, eye, bad, lack, vision, imag}"
8,"{access, anoth, order, someth, open, place}","{small, red, black, insect, dot}","{express, sentiment, sorrow}","{make, clear, condit, bright, total, thant, see}"
9,"{access, room}","{small, yellow, coat, red, insect, dot}","{emot, result, physic, exert, sensat, unpleas}","{result, confus, physic, exert, state, drug, u..."


### Calcolo punteggi similarità tra definizioni
In questa cella vengono calcolate le similarità tra le definizioni di una colonna. Il processo viene eseguito per ogni colonna e i per ognuna i valori sono salvati in una lista

In [4]:
scores = []
words = list(data.columns.values)
for word in words:
    word_scores = [similarity_score(len(def1.intersection(def2)), len(def1), len(def2))
                   for def1 in data[word] for def2 in data[word] if def1 != def2]
    scores.append(word_scores)

### Valore medio
Per sapere il valore medio di similarità tra le definizioni di una colonna si fa il rapporto tra la somma dei valori e il numero di valori. I risultati sono mostrati nell'output sottostante .
Il punteggio di similarità più alta lo si trova per la parola **'ladybug'** con un valore di 0.5967. Per i concetti generici, *'door'* e *'pain'*, i punteggi sono molto simili e sono poco sopra allo 0.20. Il punteggio più basso lo si trova per il concetto *'blurriness'* con uno score di 0.081.

In [5]:
results = [round(sum(word_scores) / len(word_scores), 4) for word_scores in scores]

print('            Astratto    Concreto')
print('Generico    ', results[2], '   ',results[0])
print('Specifico   ', results[3], '    ', results[1])

            Astratto    Concreto
Generico     0.2524     0.2345
Specifico    0.081      0.5967
